# Importing Libraries


In [6]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

### Importing predicted data and 3 minute data

In [7]:
df_pred=pd.read_csv('pred_nbeatsg.csv')
df_pred.dropna(inplace=True) #There are 6 NaN values due to WINDOW_SIZE=6
df_pred['datetime']=pd.to_datetime(df_pred['datetime'])

df_3m=pd.read_csv('../btcusdt_3m_zelta.csv')
df_3m['datetime']=pd.to_datetime(df_3m['datetime'])
df_3m.set_index('datetime',drop=True,inplace=True)

In [8]:
df_pred.head()

,datetime,open,high,low,close,volume,pred_close
6,2018-01-01 11:30:00,13539.00,13800.00,13510.00,13780.41,264.989684,13557.026
7,2018-01-01 12:30:00,13780.00,13818.55,13555.02,13570.35,292.188777,13742.600
8,2018-01-01 13:30:00,13569.98,13735.24,13400.00,13499.99,271.813553,13577.212
9,2018-01-01 14:30:00,13499.97,13670.00,13459.11,13616.99,213.095829,13524.081
10,2018-01-01 15:30:00,13632.00,13657.92,13497.98,13570.01,169.512196,13619.243


### Creating features according to the given limits.
#### On hourly data, one should idealy buy and hold for a hour if the close price is greater than 0.2% of open price
#### For predicted data, we have taken the limit as 0.5% to account for errors. This creates a stronger case to buy based on information from predicted close price

In [9]:
df_pred['True_buy']=df_pred['close']>=df_pred['open']*1.002
df_pred['pred_buy']=df_pred['pred_close']>=df_pred['open']*1.005
df_buy=df_pred[df_pred['pred_buy']]

In [10]:
df_pred['True_sell']=df_pred['close']<=df_pred['open']*0.998
df_pred['pred_sell']=df_pred['pred_close']<=df_pred['open']*0.995
df_sell=df_pred[df_pred['pred_sell']]

In [11]:
df_buy.head()

,datetime,open,high,low,close,volume,pred_close,True_buy,pred_buy
58,2018-01-03 15:30:00,14501.05,14855.00,14150.00,14455.55,1271.034018,14575.973,False,True
80,2018-01-04 14:30:00,14221.05,14410.05,13918.04,14299.01,1121.487750,14302.250,True,True
108,2018-01-05 18:30:00,15838.00,16199.00,15750.00,16093.94,996.320352,15928.647,True,True
171,2018-01-08 09:30:00,15632.48,15840.00,15510.00,15641.01,638.906874,15728.309,False,True
177,2018-01-08 15:30:00,14650.00,15183.59,14551.00,15183.59,1376.251498,14817.572,True,True


In [12]:
df_sell.head()

,datetime,open,high,low,close,volume,pred_close,True_buy,pred_buy,True_sell,pred_sell
60,2018-01-03 17:30:00,14764.73,14799.00,14400.04,14450.00,614.534955,14677.031,False,False,True,True
181,2018-01-08 19:30:00,14659.88,14693.01,13155.00,13489.95,2512.360318,14583.276,False,False,True,True
185,2018-01-08 23:30:00,14739.80,15100.00,14580.22,14900.00,723.505895,14550.065,True,False,False,True
187,2018-01-09 01:30:00,15196.56,15270.99,14527.08,14899.70,835.157357,15092.431,False,False,True,True
226,2018-01-10 16:30:00,13735.96,14128.30,13650.00,13919.08,763.192245,13657.253,True,False,False,True


#### We are getting an accuracy of 69.07% on buy side, which tells us that approximately 7 out of 10 times our buy trade is profitable.


In [13]:
sum(df_buy['pred_buy']==df_buy['True_buy'])/len(df_buy)

0.6907216494845361

#### We are getting an accuracy of 73.49% on sell side, which tells us that approximately 7 out of 10 times our short trade is profitable.

In [14]:
sum(df_sell['pred_sell']==df_sell['True_sell'])/len(df_sell)

0.7349397590361446

In [15]:
len(set(list(df_buy.index)+list(df_sell.index)))# Total number of trades

277

# Signal Generation

In [16]:
df_pred['pred_buy']=df_pred['pred_buy'].apply(lambda x: 1 if x else 0)
df_pred['pred_sell']=df_pred['pred_sell'].apply(lambda x: -1 if x else 0)

In [17]:
df_pred['trade']=df_pred['pred_buy']+df_pred['pred_sell']

In [18]:
df_pred['trade'].value_counts()

trade
 0    34925
 1      194
-1       83
Name: count, dtype: int64

In [19]:
count=0
for i,j in enumerate(df_pred['datetime']):
    try:
        if df_pred['trade'].iloc[i]==0:
            continue
        df_3m['open'].loc[j]
        df_3m['open'].loc[j+pd.DateOffset(minutes=57)]
    except:
        df_pred['trade'].iloc[i]=0
        count+=1
count

1

In [20]:
df_pred.head()

,datetime,open,high,low,close,volume,pred_close,True_buy,pred_buy,True_sell,pred_sell,trade
6,2018-01-01 11:30:00,13539.00,13800.00,13510.00,13780.41,264.989684,13557.026,True,0,False,0,0
7,2018-01-01 12:30:00,13780.00,13818.55,13555.02,13570.35,292.188777,13742.600,False,0,True,0,0
8,2018-01-01 13:30:00,13569.98,13735.24,13400.00,13499.99,271.813553,13577.212,False,0,True,0,0
9,2018-01-01 14:30:00,13499.97,13670.00,13459.11,13616.99,213.095829,13524.081,True,0,False,0,0
10,2018-01-01 15:30:00,13632.00,13657.92,13497.98,13570.01,169.512196,13619.243,False,0,True,0,0


In [21]:
df_pred['trade'].value_counts()

trade
 0    34926
 1      193
-1       83
Name: count, dtype: int64

In [22]:
df_3m.head()

,open,high,low,close,volume
datetime,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13666.11,13680.00,6.303815
2018-01-01 05:33:00,13679.98,13679.98,13554.44,13568.00,33.709206
2018-01-01 05:36:00,13568.00,13575.00,13501.01,13575.00,29.645987
2018-01-01 05:39:00,13539.99,13575.00,13400.01,13515.26,30.696039
2018-01-01 05:42:00,13515.26,13568.66,13473.47,13556.15,23.260966


In [23]:
df_3m['signals']=0

In [24]:
df_trade=df_pred[df_pred['trade']!=0]

# Creating signals in the 3 minute dataframe, where we have taken every trade for 57 minutes. 

In [25]:
for i in range(len(df_trade)):
    df_3m['signals'].loc[df_trade['datetime'].iloc[i]]=df_trade['trade'].iloc[i]
    df_3m['signals'].loc[df_trade['datetime'].iloc[i]+pd.DateOffset(minutes=57)]=-df_trade['trade'].iloc[i]

In [26]:
df_3m.head()

,open,high,low,close,volume,signals
datetime,,,,,,
2018-01-01 05:30:00,13715.65,13715.65,13666.11,13680.00,6.303815,0
2018-01-01 05:33:00,13679.98,13679.98,13554.44,13568.00,33.709206,0
2018-01-01 05:36:00,13568.00,13575.00,13501.01,13575.00,29.645987,0
2018-01-01 05:39:00,13539.99,13575.00,13400.01,13515.26,30.696039,0
2018-01-01 05:42:00,13515.26,13568.66,13473.47,13556.15,23.260966,0


In [27]:
df_3m.reset_index(inplace=True)

In [28]:
final=df_3m[['datetime','signals','open','high','low','close','volume']]

In [29]:
final.to_csv('final_logs.csv')